In [2]:
import kcwitools.io as kio
import kcwitools.utils as ku
from astropy.io import fits, ascii
import numpy as np
import matplotlib.pyplot as plt
# import matplotlib.gridspec as gridspec
from matplotlib.gridspec import GridSpec
from kcwitools.image import build_whitelight
from matplotlib.widgets import Slider, Button, TextBox, RangeSlider, RectangleSelector
# import ipywidgets as widgets

base_path = "/Users/robertseaton/Desktop/Physics-NCState/---Research/FITS data/J1429"

f1=fits.open(base_path+"/SGAS1429+1202.fits")
flux_file = fits.open(base_path+"/J1429_rb_flux.fits") # J1429_flux
# var_file = fits.open(base_path+"/J1429_var.fits") # J1429_var

hdr=f1[0].header
data=f1[0].data
wave=ku.build_wave(hdr)
wl_image=build_whitelight(hdr, data)

In [18]:
crval3 = hdr['CRVAL3']
crpix3 = hdr['CRPIX3']
cd3_3 = hdr['CD3_3']
wavedim = hdr['NAXIS3']
print("crval3:  ", crval3)
print("crpix3:  ", crpix3)
print("cd3_3:   ", cd3_3)
print("wavedim: ", wavedim)
print("wave = ",crval3," + ",cd3_3," * (np.arange(",wavedim,") + 1. - ",crpix3,")");

crval3:   3230.0
crpix3:   -270.0
cd3_3:    1.0
wavedim:  2000
wave =  3230.0  +  1.0  * (np.arange( 2000 ) + 1. -  -270.0 )


In [3]:
flux_file[0].data.size

8704000

In [4]:
#%matplotlib widget
# %matplotlib

In [5]:
# This just draws a box centered at (x,y) and sz from that center point in the n/e/s/w directions 
def plotbox(plt, x, y, labels, align, sz, c):
    for i in range(len(x)):
        plt.plot(
            [x[i]-sz, x[i]-sz, x[i]+sz, x[i]+sz, x[i]-sz], 
            [y[i]-sz, y[i]+sz, y[i]+sz, y[i]-sz, y[i]-sz], 
            '-', color=c)
        ha_ = align[i][0]
        va_ = align[i][1]
        plt.text(x[i], y[i]+1.5*sz, labels[i], color=c);


Show SGAS1429+1202 with callout boxes

In [6]:
# for j in range(wave.size):
#     print("wave[",j,"]: ",wave[j]);

In [8]:
# Show the SGAS1429 image

# %matplotlib

x_cen = [20, 25, 39, 24];
y_cen = [35, 36, 29, 21];
labels = ["1","2","3","4"];
alignment = [ # [ha, va] :: ha={'left', 'center', 'right'} va={'bottom', 'baseline', 'center', 'center_baseline', 'top'}
    ['center', 'top'],
    ['center', 'top'],
    ['center', 'top'],
    ['center', 'top']
]; 

fig = plt.figure()
grid_size = (10,6)
fig = plt.figure(figsize=(14,8))

ax0 = plt.subplot2grid(grid_size, (1, 0), colspan=2, rowspan=4)
ax = [];
ax.append(plt.subplot2grid(grid_size, (0, 2), colspan=4, rowspan=2))
ax.append(plt.subplot2grid(grid_size, (2, 2), colspan=4, rowspan=2))
ax.append(plt.subplot2grid(grid_size, (4, 2), colspan=4, rowspan=2))
ax.append(plt.subplot2grid(grid_size, (6, 2), colspan=4, rowspan=2))
# need our last box at the bottom hold our 2 range sliders and 1 "Extract" button
# ax_sliders = plt.subplot2grid(grid_size, (4, 2), colspan=4)


# source wavelength range
slower_val = 4599; # source lower bound (in angstroms)
supper_val = 4702; # source lower bound (in angstroms)
source_diff = supper_val - slower_val;

# continuum wavelength range
# Ahmed says that the continuum range needs to be roughly equal to the source range
clower_val = 4115; # continuum lower bound (in angstroms)
cupper_val = clower_val + source_diff; # continuum upper bound (in angstroms)

# Create the RangeSlider
sslider_ax = fig.add_axes([0.20, 0.2, 0.60, 0.03])
sslider = RangeSlider(sslider_ax, "Source Range:", wave.min(), wave.max(), valinit=(slower_val, supper_val))
# sslider = RangeSlider(sslider_ax, "Source Range:", 0, wave.size, valinit=(slower_val, supper_val))
print("sslider: ", sslider)
# Create the Continuum Slider
cslider_ax = fig.add_axes([0.20, 0.15, 0.60, 0.03])
cslider = RangeSlider(cslider_ax, "Continuum Range:", wave.min(), wave.max(), valinit=(clower_val, cupper_val))
# cslider = RangeSlider(cslider_ax, "Continuum Range:", 0, wave.size, valinit=(clower_val, cupper_val))

scolor = '#549c54';
ccolor = '#9c5954'; 

sbounds=[]; cbounds=[];
for j in range(4):
    sbounds.append(ax[j].axvspan(sslider.val[0], sslider.val[1], facecolor=scolor));
    cbounds.append(ax[j].axvspan(cslider.val[0], cslider.val[1], facecolor=ccolor));
    


ax[0].tick_params(top=False, bottom=False, labeltop=False, labelbottom=False)
ax[1].tick_params(top=False, bottom=False, labeltop=False, labelbottom=False)
ax[2].tick_params(top=False, bottom=False, labeltop=False, labelbottom=False)

# Create the Vertical lines on the histogram
def set_xvalues(polygon, x0, x1):
    _ndarray = polygon.get_xy()
    _ndarray[:, 0] = [x0, x0, x1, x1, x0]
    polygon.set_xy(_ndarray)
    
# print("lower_limit: ", upper_limit_line, " low: ", lower)
def update_sourceRange(val):
    print("source min: ", val[0], "  max: ", val[1])
    for j in range(4):
        set_xvalues(sbounds[j], val[0], val[1]);

    fig.canvas.draw_idle(); # Redraw the figure to ensure it updates

def update_continuumRange(val):
    print("continuum min: ", val[0], "  max: ", val[1])
    for j in range(4):
        set_xvalues(cbounds[j], val[0],val[1]);

    # Redraw the figure to ensure it updates
    fig.canvas.draw_idle()
    
# ...................................................
# Narrow-band extraction
# For each point on the arc
#   1. Calculate the average flux across the continuum range at that pixel location
#     a. sum of all the flux across the continuum range (including the endpoints)
#     b. average is sum / number of samples in sum
#   2. At each wavelength of the source spectrum, calculate:
#         adjustedSourceFlux[lambda] = SourceFlux[lambda] - ContinuumAverage
#   3. Write adjustedSourceFlux data out to FITS file
def extract_data(event):
    print("Extract data!!!");
    j = 0
    print("Source min: ", sslider.val[0], " max: ", sslider.val[1]);
    cmin = cslider.val[0];    cmax = cslider.val[1];
    print("Continuum min: ", cslider.val[0], " max: ", cslider.val[1]);
    cflux = [0]*4; cflux_N = [0]*4; cAvg = [0]*4;
    for j in range(4):
        cflux[j] = 0.0; cflux_N[j] = 0;
        print("-----")
        # for the continuum
        for i in range(0, wave.size):
            wavelength = wave[i];
            if wavelength>= cmin and wavelength<=cmax:
                cflux[j] += data[i][y_cen[j]][x_cen[j]]; 
                cflux_N[j] += 1;
                # print("i: ", i, 
                #       "  wavelength: ", wavelength, 
                #       "  flux: ", data[i][y_cen[j]][x_cen[j]], 
                #       "  sum: ", cflux[j],
                #       "  N: ", cflux_N[j]
                #  )
            
        cAvg[j] = cflux[j] / cflux_N[j];
        print("point ", j, " : ", cAvg[j]," = ",cflux[j]," / ", cflux_N[j]);    
        
    # now adjust the flux measurements inside the source range by the continuum average
    smin = sslider.val[0];    smax = sslider.val[1];
    adjusted_flux = [0.0]*4;
    for j in range(4):
        adjusted_flux[j] = 0.0; 
        print("-----")
        for i in range(0, wave.size):
            wavelength = wave[i];
            if wavelength>= smin and wavelength<=smax:
                flux_val = data[i][y_cen[j]][x_cen[j]]
                adjusted_flux[j] = flux_val - cAvg[j]; # this assumes cAvg < data; ??? is this always going to be valid???

                print("i: ", i, 
                      "  wavelength: ", wavelength, 
                      "  flux: ", data[i][y_cen[j]][x_cen[j]], 
                      "  adjusted_flux: ", adjusted_flux[j]
                 )
            


axnext = fig.add_axes([0.81, 0.05, 0.10, 0.03]) # left, bottom, width, height
bnext = Button(axnext, 'Extract')
bnext.on_clicked(extract_data)
sslider.on_changed(update_sourceRange)
cslider.on_changed(update_continuumRange)


ax0.imshow(wl_image,origin="lower",interpolation="nearest",cmap="gnuplot",vmin=0)
plotbox(ax0, x_cen, y_cen, labels, alignment, 2, 'c')

colors = ['r','g','b','y'];
j = 0
for j in range(4):
    flux = []
    for i in range(wave.size):
        # print("i: ",i)
        # print("wavelength: ",wave[i], "flux: ",data[i][y][x])
        flux.append(data[i][y_cen[j]][x_cen[j]])
    ax[j].plot(wave, flux, '-', color='black')

# def onselect(eclick, erelease):
#     print(eclick.xdata, eclick.ydata)
#     print(erelease.xdata, erelease.ydata)
# props = dict(facecolor='blue', alpha=0.5)
# rect = RectangleSelector(ax[0], onselect, interactive=True, props=props)
# # fig.show()
# rect.add_state('square')

plt.subplots_adjust(left=0, bottom=0.16, right=0.979, top=0.94, wspace=0.2, hspace=0.021)
plt.show()
    

sslider:  <matplotlib.widgets.RangeSlider object at 0x186f35e90>


In [9]:
ku.airtovac(wave)

array([3503.00360982, 3504.00412372, 3505.00463764, ..., 5501.05555568,
       5502.05608992, 5503.05662417])

In [10]:
wave

array([3502.00167624, 3503.00193315, 3504.00219007, ..., 5499.52757384,
       5500.52784093, 5501.52810801])